In [ ]:
import torch, json
from PIL import Image
from torchvision import transforms, models

In [ ]:
with open("class_names.json") as f:
    class_names = json.load(f)

num_classes = len(class_names)

map_location=torch.device('cpu')
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)
model.load_state_dict(torch.load("mobilenet_plant_weights.pt", map_location=map_location))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Define preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
def predict(image_path):
    img = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(input_tensor)
        pred = logits.argmax(1).item()
    return class_names[pred]

print("Predicted:", predict("/content/potato_late"))

Predicted: Cherry_(including_sour)___healthy


In [21]:
img = preprocess(Image.open("/content/c5617a77-0c64-404d-a25b-60338a8ec00f.jpg")).unsqueeze(0).to(device)
pred = model(img).argmax(1).item()
print("Prediction:", class_names[pred])

Prediction: Cherry_(including_sour)___healthy
